In [1]:
import sys
import warnings
import panel as pn

# DO NOT remove this line below, otherwise the app won't be found
sys.path.append('..')
warnings.filterwarnings('ignore')

In [2]:
import sim2bids
from sim2bids.sim2bids import MainArea

pn.extension('tabulator', 'ace', 'jsoneditor', 'ipywidgets', sizing_mode='stretch_width', notifications=True)

NotificationArea(sizing_mode='stretch_width')

In [4]:
path = '../output'
sys.path.append(path)

sim2bids.app.app.SoftwareVersion = 2.6
sim2bids.app.app.SoftwareRepository = 'https://github.com/the-virtual-brain/tvb-root/releases/tag/2.6'
sim2bids.app.app.SoftwareName = 'TVB'

app = MainArea()
pn.serve(app.view())

INFO:bokeh.server.server:Starting Bokeh server version 2.4.3 (running on Tornado 6.1)
INFO:bokeh.server.tornado:User authentication hooks NOT provided (default user enabled)


Launching server at http://localhost:50400


INFO:tornado.access:200 GET / (127.0.0.1) 184.00ms
INFO:tornado.access:200 GET /static/extensions/panel/bundled/jsoneditor/npm/jsoneditor@9.1.9/dist/jsoneditor.min.css (127.0.0.1) 2.00ms
INFO:tornado.access:200 GET /static/extensions/panel/bundled/notificationarea/npm/notyf@3/notyf.min.css (127.0.0.1) 2.00ms
INFO:tornado.access:200 GET /static/extensions/panel/bundled/datatabulator/tabulator-tables@4.9.3/dist/css/tabulator_simple.min.css (127.0.0.1) 4.00ms
INFO:tornado.access:200 GET /static/extensions/panel/css/alerts.css (127.0.0.1) 2.00ms
INFO:tornado.access:200 GET /static/extensions/panel/css/card.css (127.0.0.1) 6.00ms
INFO:tornado.access:200 GET /static/extensions/panel/bundled/notificationarea/ajax/libs/font-awesome/5.15.1/css/all.min.css (127.0.0.1) 9.00ms
INFO:tornado.access:200 GET /static/extensions/panel/css/dataframe.css (127.0.0.1) 11.00ms
INFO:tornado.access:200 GET /static/extensions/panel/css/json.css (127.0.0.1) 5.00ms
INFO:tornado.access:200 GET /static/extensions/p

match: []
match: []
match: []
match: []


In [4]:
path = '../output'
sys.path.append(path)

sim2bids.app.app.SoftwareVersion = 2.6
sim2bids.app.app.SoftwareRepository = 'https://github.com/the-virtual-brain/tvb-root/releases/tag/2.6'
sim2bids.app.app.SoftwareName = 'TVB'

app = MainArea()
pn.serve(app.view())

INFO:bokeh.server.server:Starting Bokeh server version 2.4.3 (running on Tornado 6.1)
INFO:bokeh.server.tornado:User authentication hooks NOT provided (default user enabled)


Launching server at http://localhost:60869


INFO:tornado.access:200 GET / (127.0.0.1) 176.00ms
INFO:tornado.access:200 GET /static/extensions/panel/bundled/jsoneditor/npm/jsoneditor@9.1.9/dist/jsoneditor.min.css (127.0.0.1) 2.00ms
INFO:tornado.access:200 GET /static/extensions/panel/bundled/datatabulator/tabulator-tables@4.9.3/dist/css/tabulator_simple.min.css (127.0.0.1) 3.00ms
INFO:tornado.access:200 GET /static/extensions/panel/bundled/notificationarea/npm/notyf@3/notyf.min.css (127.0.0.1) 2.00ms
INFO:tornado.access:200 GET /static/extensions/panel/css/alerts.css (127.0.0.1) 2.99ms
INFO:tornado.access:200 GET /static/extensions/panel/bundled/notificationarea/ajax/libs/font-awesome/5.15.1/css/all.min.css (127.0.0.1) 7.00ms
INFO:tornado.access:200 GET /static/extensions/panel/css/card.css (127.0.0.1) 8.00ms
INFO:tornado.access:200 GET /static/extensions/panel/css/dataframe.css (127.0.0.1) 2.00ms
INFO:tornado.access:200 GET /static/extensions/panel/css/debugger.css (127.0.0.1) 3.00ms
INFO:tornado.access:200 GET /static/extension

TIMES: ['bold_times.txt', 'subs_times_22min.txt', 'subs_times.txt', 'bold_times_22min.txt']
OrderedDict([('sub-01', {'distances': {'name': 'distances', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\distances.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\distances.txt', 'ext': 'txt'}, 'emp_fc': {'name': 'emp_fc', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\emp_fc.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\emp_fc.txt', 'ext': 'txt'}, 'weights': {'name': 'weights', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\weights.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\weights.txt', 'ext': 'txt'}, 'alpha-speed0.1-G10-bold_times': {'name': 'alpha-speed0.1-G10-bold_times', 'fname': 'C:\\Users\\dinar\\Desktop\\50hea

In [8]:
from collections import OrderedDict

sub = OrderedDict({'sub-01': {'distances': {'name': 'distances', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\distances.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\distances.txt', 'ext': 'txt'}, 'emp_fc': {'name': 'emp_fc', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\emp_fc.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\emp_fc.txt', 'ext': 'txt'}, 'weights': {'name': 'weights', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\weights.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\weights.txt', 'ext': 'txt'}, 'alpha-speed0.1-G10-bold_times': {'name': 'alpha-speed0.1-G10-bold_times', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\bold_times.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\bold_times.txt', 'ext': 'txt'}, 'alpha-speed0.1-G10-bold_ts': {'name': 'alpha-speed0.1-G10-bold_ts', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\bold_ts.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\bold_ts.txt', 'ext': 'txt'}, 'alpha-speed0.1-G10-subs_times': {'name': 'alpha-speed0.1-G10-subs_times', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\subs_times.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\subs_times.txt', 'ext': 'txt'}, 'alpha-speed0.1-G10-subs_ts': {'name': 'alpha-speed0.1-G10-subs_ts', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\subs_ts.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\subs_ts.txt', 'ext': 'txt'}, 'alpha-speed100-G0.0398-bold_times_22min': {'name': 'alpha-speed100-G0.0398-bold_times_22min', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\bold_times_22min.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '.', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\bold_times_22min.txt', 'ext': 'txt'}, 'alpha-speed100-G0.0398-bold_ts_22min': {'name': 'alpha-speed100-G0.0398-bold_ts_22min', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\bold_ts_22min.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\bold_ts_22min.txt', 'ext': 'txt'}, 'alpha-speed100-G0.0398-subs_times_22min': {'name': 'alpha-speed100-G0.0398-subs_times_22min', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\subs_times_22min.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '.', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\subs_times_22min.txt', 'ext': 'txt'}, 'alpha-speed100-G0.0398-subs_ts_22min': {'name': 'alpha-speed100-G0.0398-subs_ts_22min', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\subs_ts_22min.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\subs_ts_22min.txt', 'ext': 'txt'}, 'delta-speed20-G0.1-bold_times': {'name': 'delta-speed20-G0.1-bold_times', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.1\\bold_times.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.1\\bold_times.txt', 'ext': 'txt'}, 'delta-speed20-G0.1-subs_times': {'name': 'delta-speed20-G0.1-subs_times', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.1\\subs_times.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.1\\subs_times.txt', 'ext': 'txt'}, 'delta-speed20-G0.1-subs_ts': {'name': 'delta-speed20-G0.1-subs_ts', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.1\\subs_ts.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.1\\subs_ts.txt', 'ext': 'txt'}, 'delta-speed20-G0.2-bold_times': {'name': 'delta-speed20-G0.2-bold_times', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.2\\bold_times.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.2\\bold_times.txt', 'ext': 'txt'}, 'delta-speed20-G0.2-subs_times': {'name': 'delta-speed20-G0.2-subs_times', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.2\\subs_times.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.2\\subs_times.txt', 'ext': 'txt'}, 'delta-speed20-G0.2-subs_ts': {'name': 'delta-speed20-G0.2-subs_ts', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.2\\subs_ts.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.2\\subs_ts.txt', 'ext': 'txt'}, 'centres': {'name': 'centres', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\centres.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\t', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\centres.txt', 'ext': 'txt'}}})

In [9]:
sub

OrderedDict([('sub-01',
              {'distances': {'name': 'distances',
                'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\distances.txt',
                'sid': 'sub-01',
                'desc': 'default',
                'sep': '\t',
                'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\distances.txt',
                'ext': 'txt'},
               'emp_fc': {'name': 'emp_fc',
                'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\emp_fc.txt',
                'sid': 'sub-01',
                'desc': 'default',
                'sep': '\t',
                'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\emp_fc.txt',
                'ext': 'txt'},
               'weights': {'name': 'weights',
                'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\weights.txt',
                'sid': 'sub-01',
                'desc': 'default',
                'sep': '\t',
   

In [13]:
for k, v in sub['sub-01'].items():
    if 'times' in k:
        print(v, end='\n\n')

{'name': 'alpha-speed0.1-G10-bold_times', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\bold_times.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\bold_times.txt', 'ext': 'txt'}

{'name': 'alpha-speed0.1-G10-subs_times', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\subs_times.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\subs_times.txt', 'ext': 'txt'}

{'name': 'alpha-speed100-G0.0398-bold_times_22min', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\bold_times_22min.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '.', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398

INFO:bokeh.server.views.ws:WebSocket connection closed: code=1001, reason=None


In [16]:
from sim2bids.convert import convert

convert.get_specific('alpha')

['C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\bold_times.txt',
 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\bold_ts.txt',
 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\subs_times.txt',
 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\subs_ts.txt',
 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\bold_times_22min.txt',
 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\bold_ts_22min.txt',
 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\subs_times_22min.txt',
 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\subs_ts_22min.txt']

In [17]:
from sim2bids.app import app

def get_times(rhythm, ftype='bold_times'):
    content = []
    
    for file in app.ALL_FILES:
        if rhythm in file and ftype in file:
            content.append(file)
    
    return content

In [19]:
get_times('delta', 'bold_times')

['C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.1\\bold_times.txt',
 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.2\\bold_times.txt']

In [24]:
import pandas as pd

a1 = pd.read_csv('C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.1\\bold_times.txt',
                header=None, sep='\t')
a2 = pd.read_csv('C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.2\\bold_times.txt',
                header=None, sep='\t')

In [28]:
a1.equals(a2)

True

In [50]:
def check_times(subs):
    # 1. get rhythm type
    rhythm = re.findall(r'alpha|delta|gamma|theta|beta', subs['path'])[0]
    
    # 2. get 'times' type
    times_type = [t for t in app.TIMES if t in subs['path']]
    
    if isinstance(times_type, list):
        # 3. get all files of specific type 
        files = get_times(rhythm, times_type[0])
        
        # 4. if there are multiple versions, 
        if len(files) == 1:
            
    

In [54]:
check_times({'name': 'delta-speed20-G0.2-subs_times', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.2\\subs_times.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '\n', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.2\\subs_times.txt', 'ext': 'txt'})




['C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.1\\subs_times.txt', 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_delta\\cspeed20\\csf 0.2\\subs_times.txt']


In [41]:
app.TIMES

['bold_times.txt',
 'subs_times_22min.txt',
 'subs_times.txt',
 'bold_times_22min.txt']

In [52]:
check_times({'name': 'alpha-speed100-G0.0398-bold_times_22min', 'fname': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\bold_times_22min.txt', 'sid': 'sub-01', 'desc': 'default', 'sep': '.', 'path': 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\bold_times_22min.txt', 'ext': 'txt'})

['C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\bold_times_22min.txt']


In [55]:
from scipy.io import loadmat

loadmat('C:\\Users\\dinar\\Desktop\\50healthy_demo\\AY_20111004\\series_alpha\\cspeed100\\csf 0.0325\\AY_20111004_SC_new.mat_SC_log_SC_22min_csf_0.0325.mat')




{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Tue Feb 14 04:48:46 2017',
 '__version__': '1.0',
 '__globals__': [],
 'Bold_time': array([[5.0000e+02, 1.0000e+03, 1.5000e+03, ..., 1.3190e+06, 1.3195e+06,
         1.3200e+06]]),
 'Bold_data': array([[[[   31.30501829],
          [   22.94625814],
          [ -110.86465291],
          ...,
          [   23.94189774],
          [   57.64111017],
          [  -74.77319948]]],
 
 
        [[[   29.59338953],
          [   21.29085872],
          [ -112.49132285],
          ...,
          [   22.44538672],
          [   55.95230906],
          [  -76.48548574]]],
 
 
        [[[   18.65865173],
          [   10.86446101],
          [ -122.97435788],
          ...,
          [   12.98478864],
          [   45.34490782],
          [  -87.68838787]]],
 
 
        ...,
 
 
        [[[-1470.05512465],
          [-1400.31684914],
          [-1562.75033049],
          ...,
          [-1218.97335231],
          [-1303.16777743],
 

In [60]:
from sim2bids.generate import subjects as subs

subs.find_matches(['C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\bold_times.txt'])

[]

In [62]:
re.findall(r'[A-Za-z]{2,3}_[0-9]{2,}', 'C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed0.1\\csf 10\\bold_times.txt')





['AA_20120815']

In [10]:
split = ['bold', 'times', '22min']

[split[-idx] for idx in list(range(3, 0, -1))]

['bold', 'times', '22min']

In [6]:
from scipy.io import loadmat

mat = loadmat('C:\\Users\\dinar\\Desktop\\50healthy_demo\\AA_20120815\\series_alpha\\cspeed100\\csf 0.0398\\YE_20120910_SC.mat_csf_0.0398_speed_100_22min.mat')






In [14]:
one = mat['subs_data'][:, 0, :].shape

In [15]:
two = mat['subs_data'][:, 1, :].shape

In [16]:
one == two

True

In [10]:
mat['Bold_data'][:, 0, :, 0].shape

(600, 68)

In [4]:
import re

re.findall(r'[0-9]+min', 'YE_20120910_SC.mat_csf_0.0398_speed_100_22min')

['22min']

INFO:bokeh.server.views.ws:WebSocket connection closed: code=1001, reason=None


In [4]:
rhythms = pn.Column('#### Specify rhythms', pn.widgets.CheckBoxGroup(
    value=[], options=['Alpha', 'Beta', 'Delta', 'Gamma', 'Theta'],
    inline=True, margin=(-10,0,0,10)))

speed = pn.Column('#### Specify `conduction speed` folder name', pn.widgets.TextInput(value='cspeed', margin=(-10,0,0,0)))

global_coupling = pn.Column('#### Specify `global coupling scaling factor` folder name', 
                            pn.widgets.TextInput(value='csf', margin=(-10,0,0,0)))

col = pn.Column('### Specify simulation information', rhythms, speed, global_coupling)

In [5]:
pn.serve(col)

INFO:bokeh.server.server:Starting Bokeh server version 2.4.3 (running on Tornado 6.1)
INFO:bokeh.server.tornado:User authentication hooks NOT provided (default user enabled)


Launching server at http://localhost:50198


INFO:tornado.access:200 GET / (127.0.0.1) 123.00ms
INFO:tornado.access:200 GET /static/extensions/panel/bundled/jsoneditor/npm/jsoneditor@9.1.9/dist/jsoneditor.min.css (127.0.0.1) 2.00ms
INFO:tornado.access:200 GET /static/extensions/panel/bundled/datatabulator/tabulator-tables@4.9.3/dist/css/tabulator_simple.min.css (127.0.0.1) 2.00ms
INFO:tornado.access:200 GET /static/extensions/panel/bundled/notificationarea/npm/notyf@3/notyf.min.css (127.0.0.1) 4.00ms
INFO:tornado.access:200 GET /static/extensions/panel/bundled/notificationarea/ajax/libs/font-awesome/5.15.1/css/all.min.css (127.0.0.1) 6.01ms
INFO:tornado.access:200 GET /static/extensions/panel/css/alerts.css (127.0.0.1) 2.00ms
INFO:tornado.access:200 GET /static/extensions/panel/css/card.css (127.0.0.1) 4.00ms
INFO:tornado.access:200 GET /static/extensions/panel/css/debugger.css (127.0.0.1) 2.00ms
INFO:tornado.access:200 GET /static/extensions/panel/css/dataframe.css (127.0.0.1) 4.00ms
INFO:tornado.access:200 GET /static/extension